# Create a df with info per patient instead of per slice

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
raw_folder = "../../LVNC_dataset/raw_dataset/"
df = pd.read_pickle(os.path.join(raw_folder, "df_info.pick"))

In [3]:
df

,patient,slice,set,reversed,score,pta_qlvthc,pta_class_map
0,ANH,4,Hebron,False,5.0,41.98,41.801974
1,ANH,5,Hebron,False,5.0,42.02,41.825516
2,ANH,6,Hebron,False,4.5,42.78,42.629004
3,ANH,7,Hebron,False,4.5,38.60,38.458082
4,ANH,8,Hebron,False,5.0,37.26,37.127519
...,...,...,...,...,...,...,...
3077,X9,4,X,False,5.0,35.32,35.079389
3078,X9,5,X,False,5.0,30.86,30.661447
3079,X9,6,X,False,5.0,26.88,26.719278
3080,X9,7,X,False,5.0,25.68,25.507652


## Get max slice for every patient

In [4]:
excel_scores = "../../lvnc-dataset/resumen-datos-npP1-P307-v3.xlsx"
excel_outputs = "../../lvnc-dataset/salidas-datosP1-P307-v3.xlsx"

In [5]:
def convert_cell(x, first_char: str):
    """
    Function to process and transform the cells of the column containing the patients or the slices
    """
    if isinstance(x, str):
        if len(x)>0:
            if x[0]==first_char:
                if "®" in x:
                    return x.split(" ")[0] + "-REVERSED"
                return x.split(" ")[0]
    return np.NaN   

In [6]:
def read_excel_scores(sheet_name: str, first_char_patient: str):
    df_excel = pd.read_excel(excel_scores, sheet_name=sheet_name, skiprows=17, usecols="A,C,D",
                             header=None, names=['patient', 'slice', 'score'], #index_col=[0,1],
                             converters={
                                 'patient': lambda x: convert_cell(x, first_char_patient),
                                 'slice': lambda x: convert_cell(x, "c")
                             })#.reset_index()
    df_excel['patient'] = df_excel['patient'].ffill()
    df_excel = df_excel.dropna(subset=['slice'])
    print(f"Total number of slices: {len(df_excel)}")
    df_excel["slice"] = df_excel["slice"].apply(lambda x: int(x[1:])) # Remove initial 'c'
    # Invert the order of "reversed" slices
    aux_dfs = []
    for patient, group in df_excel.groupby('patient'):
        if "-REVERSED" in patient:
            n_patient = patient.replace("-REVERSED", "")
            max_slice = max(group["slice"])
            group["patient"]=n_patient
            #group["slice"]=group["slice"].apply(lambda x: 1 + max_slice - x)
            #group.sort_values(by=["slice"], inplace=True)
            group["reversed"] = True
        else:
            group["reversed"] = False
            
        aux_dfs.append(group)
    
    df_excel = pd.concat(aux_dfs)
    return df_excel

In [7]:
df_max_slice_p = read_excel_scores("P1-P307", "P").groupby("patient")["slice"].agg(max)
df_max_slice_x = read_excel_scores("X1-X67", "X").groupby("patient")["slice"].agg(max)
df_h = read_excel_scores("ValldHbron", "P")
df_h["patient"]=df_h["patient"].apply(lambda x: x.split("-")[1])
df_max_slice_h = df_h.groupby("patient")["slice"].agg(max)

<ipython-input-6-bfd8aaa62aad>:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_excel = pd.read_excel(excel_scores, sheet_name=sheet_name, skiprows=17, usecols="A,C,D",


Total number of slices: 2939


<ipython-input-6-bfd8aaa62aad>:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_excel = pd.read_excel(excel_scores, sheet_name=sheet_name, skiprows=17, usecols="A,C,D",


Total number of slices: 564


<ipython-input-6-bfd8aaa62aad>:2: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_excel = pd.read_excel(excel_scores, sheet_name=sheet_name, skiprows=17, usecols="A,C,D",


Total number of slices: 283


In [8]:
df_max_slice = pd.concat([df_max_slice_p, df_max_slice_x, df_max_slice_h])
df_max_slice = df_max_slice.rename("max_slice")

## Get VT% from the spreadsheets

In [9]:
df_vol_p = pd.read_excel(excel_outputs, skiprows=15)
df_vol_p = df_vol_p[["Paciente", "VT%"]]
df_vol_p = df_vol_p[df_vol_p["VT%"].notna()]
df_vol_p["Paciente"] = df_vol_p["Paciente"].apply(lambda x: x.split(" ")[0])
df_vol_p["VT%"] = df_vol_p["VT%"].apply(lambda x: float(x.replace(",", ".")) if isinstance(x, str) else x)

df_vol_x = pd.read_excel(excel_outputs, skiprows=15, sheet_name="X1-X67")
df_vol_x = df_vol_x[["Paciente", "corte","VT%"]]
df_vol_x["Paciente"] = df_vol_x["Paciente"].ffill(limit=5)
df_vol_x = df_vol_x[df_vol_x["corte"].notna()]
df_vol_x = df_vol_x[df_vol_x["VT%"].notna()]
df_vol_x = df_vol_x[df_vol_x["Paciente"].notna()]
del df_vol_x["corte"]
df_vol_x["VT%"] = df_vol_x["VT%"].apply(lambda x: float(x.replace(",", ".")) if isinstance(x, str) else x)

df_vol_h = pd.read_excel(excel_outputs, skiprows=15, sheet_name="ValldHbron")
df_vol_h = df_vol_h[["Paciente", "corte","VT%"]]
df_vol_h["Paciente"] = df_vol_h["Paciente"].ffill()
df_vol_h = df_vol_h[df_vol_h["corte"].notna()]
df_vol_h = df_vol_h[df_vol_h["VT%"].notna()]
df_vol_h = df_vol_h[df_vol_h["Paciente"].notna()]
df_vol_h["Paciente"] = df_vol_h["Paciente"].apply(lambda x: x.split("-")[1])
del df_vol_h["corte"]
df_vol_h["VT%"] = df_vol_h["VT%"].apply(lambda x: float(x.replace(",", ".")) if isinstance(x, str) else x)

df_vol = pd.concat([df_vol_p, df_vol_x, df_vol_h])
df_vol = df_vol.rename(columns={"Paciente": "patient"})

<ipython-input-9-672636cd22e3>:1: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_vol_p = pd.read_excel(excel_outputs, skiprows=15)
<ipython-input-9-672636cd22e3>:7: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_vol_x = pd.read_excel(excel_outputs, skiprows=15, sheet_name="X1-X67")
<ipython-input-9-672636cd22e3>:16: FutureWarning: Your version of xlrd is 1.2.0. In xlrd >= 2.0, only the xls format is supported. As a result, the openpyxl engine will be used if it is installed and the engine argument is not specified. Install openpyxl instead.
  df_vol_h = pd.read_excel(excel_outputs, skiprows=15, sheet_name="ValldHbron")

In [10]:
df_vol[df_vol["VT%"]>100]

,patient,VT%
578,X50,21161.0
777,X67,35106.0


There has been a problem reading these 2 values (probably because of the usage of `,` instead of `.` for decimals), so we fix this:

In [11]:
df_vol.loc[df_vol["VT%"]>100, "VT%"] = df_vol.loc[df_vol["VT%"]>100, "VT%"]/1000
df_vol[df_vol["VT%"]>100]

,patient,VT%


## Create new dataframe with patient information

In [11]:
is_reversed = df.groupby("patient")["reversed"].sum()>0

In [14]:
df_set = df.groupby("patient")["set"].apply(lambda df: df.unique()[0])

In [13]:
df_patients = pd.merge(df_vol, df_max_slice, on="patient")
df_patients = df_patients.merge(is_reversed, on="patient")
df_patients = df_patients.merge(df_set, on="patient")

In [14]:
df_patients

,patient,VT%,max_slice,reversed
0,P1,29.7443,7,False
1,P2,33.0554,7,False
2,P3,29.0985,8,False
3,P4,22.7398,7,False
4,P5,28.5148,9,False
...,...,...,...,...
374,MEAP,29.0362,9,True
375,MJGV,32.2028,11,True
376,RAM,29.1448,5,True
377,RGP,36.3299,10,True


## Build df with patient info

In [15]:
df_patients = pd.merge(df_patients, df.groupby("patient")["slice"].count(), on="patient")
df_patients = df_patients.rename(columns={"slice": "num_slices"})
df_patients = pd.merge(df_patients, df.groupby("patient")["slice"].agg(list), on="patient")
df_patients = df_patients.rename(columns={"slice": "slices"})
df_patients["slices_left"] = df_patients.apply(lambda row: list(set(range(1,row["max_slice"]+1, 1))-set(row["slices"])), axis=1)
df_patients

,patient,VT%,max_slice,reversed,num_slices,slices,slices_left
0,P1,29.7443,7,False,7,"[1, 2, 3, 4, 5, 6, 7]",[]
1,P2,33.0554,7,False,7,"[1, 2, 3, 4, 5, 6, 7]",[]
2,P3,29.0985,8,False,7,"[2, 3, 4, 5, 6, 7, 8]",[1]
3,P4,22.7398,7,False,7,"[1, 2, 3, 4, 5, 6, 7]",[]
4,P5,28.5148,9,False,9,"[2, 3, 4, 5, 6, 7, 8, 9, 10]",[1]
...,...,...,...,...,...,...,...
374,MEAP,29.0362,9,True,7,"[3, 4, 5, 6, 7, 8, 9]","[1, 2]"
375,MJGV,32.2028,11,True,5,"[7, 8, 9, 10, 11]","[1, 2, 3, 4, 5, 6]"
376,RAM,29.1448,5,True,4,"[2, 3, 4, 5]",[1]
377,RGP,36.3299,10,True,8,"[3, 4, 5, 6, 7, 8, 9, 10]","[1, 2]"


In [16]:
# Patients with slices outside the boundaries defined by max_slice
# It seems that there is only one patient (P5) 
(df_patients.apply(lambda row: list(set(row["slices"])-set(range(1,row["max_slice"]+1, 1))), axis=1).apply(len)>0).sum()

1

In [17]:
df_patients.loc[df_patients["patient"]=="P5","max_slice"]=10

It is patient P5 that has score for slice 10 (not present in the excel file). However, the output is a 5 according to cardioligists, so we are keeping it.

## Double check `reversed` field

I will consider "reversed" de patients whose slices go from the base to the apex.

Here I will mark petients whose `reversed` field is wrong according to that rule.

Reversed non-problematic patients (checked manually patient by patient):

In [18]:
reversed_np_patients = [
    "CAC", "FAAJ", "FDC", "GCS", "IIP", "JADP", "JDB", "MEAP", "MJGV", "P155", "P156", "P158", "P166", "P169",
    "P174", "P179", "P182", "P186", "P190", "P192", "P199", "P209", "P211", "P213", "P217", "P225", "P226",
    "P227", "P232", "P236", "P240", "P244", "P247", "P259", "P273", "P281", "P282", "P290", "P291", "P292",
    "RAM", "RGP", "TGR", "X5", "X6", "X7", "X8", "X15", "X18", "X21", "X31", "X32", "X39", "X41", "X44",
    "X46", "X50", "X54", "X59", "X61", "X62"
]

In [19]:
assert set(reversed_np_patients) == set(df_patients[df_patients["reversed"]]["patient"])

Problematic patients:

In [20]:
problematic_patients = [
    "P221", 
]

In [21]:
for p in problematic_patients:
    r = df_patients.loc[df_patients["patient"]==p, "reversed"].all()
    print("Patient", p, "is marked as", "reversed" if r else "not reversed")

Patient P221 is marked as not reversed


However, I think that it should be marked as reversed, so I am changing it:

In [22]:
df_patients.loc[df_patients["patient"].isin(problematic_patients), "reversed"] = np.logical_not(df_patients.loc[df_patients["patient"].isin(problematic_patients), "reversed"])

In [23]:
for p in problematic_patients:
    r = df_patients.loc[df_patients["patient"]==p, "reversed"].all()
    print("Patient", p, "is marked as", "reversed" if r else "not reversed")

Patient P221 is marked as reversed


The rest of the patients are fine.

## Save the dataframe

In [24]:
pd.to_pickle(df_patients, os.path.join(raw_folder, "df_info_patients.pick"))